# 알바공고별 상세페이지 크롤링 작업

In [1]:
#-*- coding:utf-8 -*-

from bs4 import BeautifulSoup
import urllib.request
import csv

In [4]:
def alba_info_get(alba_url):
    alba_info = [""]*15
    
    try:
        with urllib.request.urlopen(alba_url) as url:  
            doc = url.read() 
        alba_doc = BeautifulSoup(doc, "html.parser")   
        s_alba = alba_doc.find('div', class_ = 'wrapContents')
        try:
            s_company = s_alba.find('p', class_="workcomnm").find('strong').text.strip()
        except:
            s_company = s_alba.find('p', class_="logo").text.strip()
        s_title = s_alba.find('p', class_= "detailTitle").text.strip()

        alba_info[0] = alba_url #url 주소
        alba_info[1] = s_company #회사 이름
        alba_info[2] = s_title #공고 제목
        
        hired_type = s_alba.find('li', class_="recruitOption_K").text.replace(',', '').split()[1:]

        category = []
        jobkind_list = s_alba.find('li', class_="jobKind").find_all('li')
        for li in jobkind_list:
            jobkinds = li.find_all('a')
            category.append(jobkinds[-1].text.strip())
        
        alba_info[3] = hired_type #고용형태
        alba_info[4] = category #업직종

        #연수생 교육생이 아닌경우 아래를 수행한다. 
        if "연수생" not in alba_info[3] and "교육생" not in alba_info[3]:
            hired_time = s_alba.find('li', class_="workperiodcd").find('a').text.strip()
            work_time = s_alba.find('li', class_="worktimecd").text.replace("근무시간", "").replace(' ', '').split('~')
            start_time = float(work_time[0][0:2]) + (float(work_time[0][3:5])/60)
            end_time = float(work_time[1][0:2]) + (float(work_time[1][3:5])/60)
            alba_info[13] = start_time
            alba_info[14] = end_time
            
            nightwork = False
            if start_time<6.0 or start_time>=22.0:
                nightwork = True
            if end_time<=6.0 or end_time>22.0:
                nightwork = True
            if end_time<start_time:
                end_time += 24.0
                nightwork = True
            total_time = end_time - start_time
            
            alba_info[5] = hired_time
            alba_info[6] = total_time
            alba_info[7] = nightwork

        else:
            alba_info[5] = ""
            alba_info[6] = 0
            alba_info[13] = None
            alba_info[14] = None
            alba_info[7] = False
            
        juso = s_alba.find('li', class_='address divide').find('strong').text.split('\xa0')[0].strip()
        alba_info[8]= juso #진짜 주소 

        #사이트 소스 중 아래쪽에 위치한 json파일 가져오기
        json_source = alba_doc.find_all("script", type = "text/javascript")
        for j in json_source:
            if 'mapDetailUrl' in j.text:
                for part in j.text.split(';'):
                    if 'FrameDaum' in part:
                        json_map = part

        for i in json_map.split('&'):
            if 'lngx' in i:
                lngx= float(i.split('=')[-1])
            if 'laty' in i:
                laty= float(i.split('=')[-1])
                                                                
        
        
        alba_info[9] = laty #위도
        alba_info[10] = lngx #경도
        alba_info[11] = None #x좌표(알바천국은 제공안함) 
        alba_info[12] = None #y좌표(알바천국은 제공안함)

        # [url, 기업, 알바명, 고용형태, 직업종류, 고용기간, 실근무시간, 야간근무여부, 상세주소, 위도, 경도, x좌표, y좌표, 시작시간, 끝시간]
        return alba_info
    
    except:
        return alba_info

In [5]:
alba_info_get("http://www.alba.co.kr/job/Detail.asp?adid=97433992")

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

# 알바 주소 목록에서 정보를 가져와서 같은 데이터프레임으로 만들기

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

#원본파일:
original_csv = "albaheaven_info_1215"

df = pd.read_csv(original_csv+ ".csv")

In [7]:
df['주소']

0       http://www.alba.co.kr/job/Detail.asp?adid=97438965
1       http://www.alba.co.kr/job/Detail.asp?adid=97436358
2       http://www.alba.co.kr/job/Detail.asp?adid=97431487
3       http://www.alba.co.kr/job/Detail.asp?adid=97360731
4       http://www.alba.co.kr/job/Detail.asp?adid=97360734
5       http://www.alba.co.kr/job/Detail.asp?adid=97360733
6       http://www.alba.co.kr/job/Detail.asp?adid=97360732
7       http://www.alba.co.kr/job/Detail.asp?adid=97438911
8       http://www.alba.co.kr/job/Detail.asp?adid=97438906
9       http://www.alba.co.kr/job/Detail.asp?adid=97360652
10      http://www.alba.co.kr/job/Detail.asp?adid=97360650
11      http://www.alba.co.kr/job/Detail.asp?adid=97431678
12      http://www.alba.co.kr/job/Detail.asp?adid=97433992
13      http://www.alba.co.kr/job/Detail.asp?adid=97344717
14      http://www.alba.co.kr/job/Detail.asp?adid=97360139
15      http://www.alba.co.kr/job/Detail.asp?adid=97360140
16      http://www.alba.co.kr/job/Detail.asp?adid=973601

In [8]:
df.head()

,알바명,알바설명,지역,지역세부,급여종류,금액,시간,주소
0,와이씨 컴퍼니(직영5팀),"홈쇼핑/택배 야간물류센터 상,하차 인원모집 최대15만원 당일지급",대구,중구,일급,153732,19:00~05:00,http://www.alba.co.kr/job/Detail.asp?adid=97438965
1,(주)한성엠에스,[12/16~20 주간] 콤프케이스 생산 단기 아르바이트(12/27지급),경남,창원시,일급,100000,08:00~19:30,http://www.alba.co.kr/job/Detail.asp?adid=97436358
2,(주)이학,이학갈비 연수점 대학생 아르바이트,인천,연수구,시급,8350,12:00~21:00,http://www.alba.co.kr/job/Detail.asp?adid=97431487
3,호텔신라,"[시급10,308원] 신라호텔 설특판 콜센터 근무자 모집(주휴포함)",서울,중구,시급,10308,09:00~18:00,http://www.alba.co.kr/job/Detail.asp?adid=97360731
4,호텔신라,"[시급10,308원] 신라호텔 설특판 콜센터 근무자 모집(주휴포함)",서울,강남구,시급,10308,09:00~18:00,http://www.alba.co.kr/job/Detail.asp?adid=97360734


In [11]:
# colname = [url주소, 기업, 알바명, 고용형태, 직업종류, 고용기간, 실근무시간, 야간근무여부, 상세주소, 위도, 경도, x좌표, y좌표, 근무시작시간, 근무끝시간]
import time

f = open(original_csv + "_specific.csv", 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
wr.writerow(['url주소', '기업', '알바명', '고용형태', '직업종류', '고용기간', '실근무시간', '야간근무여부', '상세주소', '위도', '경도', 'x좌표', 'y좌표', 'starttime', 'endtime'])

for i in range(len(df)):
    url = df['주소'][i]
    info_list = alba_info_get(url)
    print(url)
    wr.writerow([url, info_list[1], info_list[2], info_list[3], info_list[4], info_list[5], info_list[6], info_list[7], info_list[8], info_list[9], info_list[10], info_list[11], info_list[12], info_list[13], info_list[14]]) 
    
    rand_value = randint(1, 10)
    time.sleep(rand_value)

f.close()

http://www.alba.co.kr/job/Detail.asp?adid=97438965
http://www.alba.co.kr/job/Detail.asp?adid=97436358
http://www.alba.co.kr/job/Detail.asp?adid=97431487
http://www.alba.co.kr/job/Detail.asp?adid=97360731
http://www.alba.co.kr/job/Detail.asp?adid=97360734
http://www.alba.co.kr/job/Detail.asp?adid=97360733
http://www.alba.co.kr/job/Detail.asp?adid=97360732
http://www.alba.co.kr/job/Detail.asp?adid=97438911
http://www.alba.co.kr/job/Detail.asp?adid=97438906
http://www.alba.co.kr/job/Detail.asp?adid=97360652
http://www.alba.co.kr/job/Detail.asp?adid=97360650
http://www.alba.co.kr/job/Detail.asp?adid=97431678
http://www.alba.co.kr/job/Detail.asp?adid=97433992
http://www.alba.co.kr/job/Detail.asp?adid=97344717
http://www.alba.co.kr/job/Detail.asp?adid=97360139
http://www.alba.co.kr/job/Detail.asp?adid=97360140
http://www.alba.co.kr/job/Detail.asp?adid=97360143
http://www.alba.co.kr/job/Detail.asp?adid=97344461
http://www.alba.co.kr/job/Detail.asp?adid=97431680
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97390179
http://www.alba.co.kr/job/Detail.asp?adid=97390164
http://www.alba.co.kr/job/Detail.asp?adid=97420290
http://www.alba.co.kr/job/Detail.asp?adid=97440247
http://www.alba.co.kr/job/Detail.asp?adid=97439869
http://www.alba.co.kr/job/Detail.asp?adid=97434782
http://www.alba.co.kr/job/Detail.asp?adid=97435974
http://www.alba.co.kr/job/Detail.asp?adid=97392539
http://www.alba.co.kr/job/Detail.asp?adid=97420788
http://www.alba.co.kr/job/Detail.asp?adid=97429943
http://www.alba.co.kr/job/Detail.asp?adid=97413162
http://www.alba.co.kr/job/Detail.asp?adid=97435123
http://www.alba.co.kr/job/Detail.asp?adid=97399792
http://www.alba.co.kr/job/Detail.asp?adid=97399784
http://www.alba.co.kr/job/Detail.asp?adid=97399779
http://www.alba.co.kr/job/Detail.asp?adid=97434519
http://www.alba.co.kr/job/Detail.asp?adid=97399767
http://www.alba.co.kr/job/Detail.asp?adid=97399761
http://www.alba.co.kr/job/Detail.asp?adid=97388064
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97418869
http://www.alba.co.kr/job/Detail.asp?adid=97432759
http://www.alba.co.kr/job/Detail.asp?adid=97432416
http://www.alba.co.kr/job/Detail.asp?adid=97431701
http://www.alba.co.kr/job/Detail.asp?adid=97418896
http://www.alba.co.kr/job/Detail.asp?adid=97430416
http://www.alba.co.kr/job/Detail.asp?adid=97430306
http://www.alba.co.kr/job/Detail.asp?adid=97430307
http://www.alba.co.kr/job/Detail.asp?adid=97431303
http://www.alba.co.kr/job/Detail.asp?adid=97431604
http://www.alba.co.kr/job/Detail.asp?adid=97430153
http://www.alba.co.kr/job/Detail.asp?adid=97431544
http://www.alba.co.kr/job/Detail.asp?adid=97430057
http://www.alba.co.kr/job/Detail.asp?adid=97431530
http://www.alba.co.kr/job/Detail.asp?adid=97430163
http://www.alba.co.kr/job/Detail.asp?adid=97429932
http://www.alba.co.kr/job/Detail.asp?adid=96978912
http://www.alba.co.kr/job/Detail.asp?adid=96985437
http://www.alba.co.kr/job/Detail.asp?adid=97419625
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97410604
http://www.alba.co.kr/job/Detail.asp?adid=97410607
http://www.alba.co.kr/job/Detail.asp?adid=97410612
http://www.alba.co.kr/job/Detail.asp?adid=97410603
http://www.alba.co.kr/job/Detail.asp?adid=97410602
http://www.alba.co.kr/job/Detail.asp?adid=97410611
http://www.alba.co.kr/job/Detail.asp?adid=97410240
http://www.alba.co.kr/job/Detail.asp?adid=97346870
http://www.alba.co.kr/job/Detail.asp?adid=97346863
http://www.alba.co.kr/job/Detail.asp?adid=97346858
http://www.alba.co.kr/job/Detail.asp?adid=97417683
http://www.alba.co.kr/job/Detail.asp?adid=97346850
http://www.alba.co.kr/job/Detail.asp?adid=97346835
http://www.alba.co.kr/job/Detail.asp?adid=97411591
http://www.alba.co.kr/job/Detail.asp?adid=97436253
http://www.alba.co.kr/job/Detail.asp?adid=97436175
http://www.alba.co.kr/job/Detail.asp?adid=97436138
http://www.alba.co.kr/job/Detail.asp?adid=97435673
http://www.alba.co.kr/job/Detail.asp?adid=97435513
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97417931
http://www.alba.co.kr/job/Detail.asp?adid=97417922
http://www.alba.co.kr/job/Detail.asp?adid=97417901
http://www.alba.co.kr/job/Detail.asp?adid=97417830
http://www.alba.co.kr/job/Detail.asp?adid=97417835
http://www.alba.co.kr/job/Detail.asp?adid=97417705
http://www.alba.co.kr/job/Detail.asp?adid=97417541
http://www.alba.co.kr/job/Detail.asp?adid=97417482
http://www.alba.co.kr/job/Detail.asp?adid=97417472
http://www.alba.co.kr/job/Detail.asp?adid=97417428
http://www.alba.co.kr/job/Detail.asp?adid=97417406
http://www.alba.co.kr/job/Detail.asp?adid=97417382
http://www.alba.co.kr/job/Detail.asp?adid=97341437
http://www.alba.co.kr/job/Detail.asp?adid=97371798
http://www.alba.co.kr/job/Detail.asp?adid=97416953
http://www.alba.co.kr/job/Detail.asp?adid=97416032
http://www.alba.co.kr/job/Detail.asp?adid=97415925
http://www.alba.co.kr/job/Detail.asp?adid=97415848
http://www.alba.co.kr/job/Detail.asp?adid=97415760
http://www.alba.co.kr/job/Detai

...

http://www.alba.co.kr/job/Detail.asp?adid=97254409
http://www.alba.co.kr/job/Detail.asp?adid=97279230
http://www.alba.co.kr/job/Detail.asp?adid=97278112
http://www.alba.co.kr/job/Detail.asp?adid=97278064
http://www.alba.co.kr/job/Detail.asp?adid=97278041
http://www.alba.co.kr/job/Detail.asp?adid=97277842
http://www.alba.co.kr/job/Detail.asp?adid=97260799
http://www.alba.co.kr/job/Detail.asp?adid=97254381
http://www.alba.co.kr/job/Detail.asp?adid=97276649
http://www.alba.co.kr/job/Detail.asp?adid=97275508
http://www.alba.co.kr/job/Detail.asp?adid=97275507
http://www.alba.co.kr/job/Detail.asp?adid=97275505
http://www.alba.co.kr/job/Detail.asp?adid=97275506
http://www.alba.co.kr/job/Detail.asp?adid=97274744
http://www.alba.co.kr/job/Detail.asp?adid=97274734
http://www.alba.co.kr/job/Detail.asp?adid=97274725
http://www.alba.co.kr/job/Detail.asp?adid=97268360
http://www.alba.co.kr/job/Detail.asp?adid=97271642
http://www.alba.co.kr/job/Detail.asp?adid=97273700
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97180639
http://www.alba.co.kr/job/Detail.asp?adid=97221433
http://www.alba.co.kr/job/Detail.asp?adid=97180617
http://www.alba.co.kr/job/Detail.asp?adid=97226039
http://www.alba.co.kr/job/Detail.asp?adid=97208931
http://www.alba.co.kr/job/Detail.asp?adid=97180610
http://www.alba.co.kr/job/Detail.asp?adid=97220879
http://www.alba.co.kr/job/Detail.asp?adid=97180595
http://www.alba.co.kr/job/Detail.asp?adid=97220633
http://www.alba.co.kr/job/Detail.asp?adid=97180569
http://www.alba.co.kr/job/Detail.asp?adid=97208921
http://www.alba.co.kr/job/Detail.asp?adid=97219686
http://www.alba.co.kr/job/Detail.asp?adid=97251692
http://www.alba.co.kr/job/Detail.asp?adid=97208790
http://www.alba.co.kr/job/Detail.asp?adid=96948900
http://www.alba.co.kr/job/Detail.asp?adid=97180545
http://www.alba.co.kr/job/Detail.asp?adid=97214510
http://www.alba.co.kr/job/Detail.asp?adid=97214500
http://www.alba.co.kr/job/Detail.asp?adid=97214494
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97221766
http://www.alba.co.kr/job/Detail.asp?adid=97219348
http://www.alba.co.kr/job/Detail.asp?adid=97220984
http://www.alba.co.kr/job/Detail.asp?adid=97220885
http://www.alba.co.kr/job/Detail.asp?adid=97220736
http://www.alba.co.kr/job/Detail.asp?adid=97200860
http://www.alba.co.kr/job/Detail.asp?adid=97212140
http://www.alba.co.kr/job/Detail.asp?adid=97217769
http://www.alba.co.kr/job/Detail.asp?adid=97217705
http://www.alba.co.kr/job/Detail.asp?adid=97217679
http://www.alba.co.kr/job/Detail.asp?adid=97217649
http://www.alba.co.kr/job/Detail.asp?adid=97217598
http://www.alba.co.kr/job/Detail.asp?adid=97217572
http://www.alba.co.kr/job/Detail.asp?adid=97217532
http://www.alba.co.kr/job/Detail.asp?adid=97217498
http://www.alba.co.kr/job/Detail.asp?adid=97217399
http://www.alba.co.kr/job/Detail.asp?adid=97216925
http://www.alba.co.kr/job/Detail.asp?adid=97216084
http://www.alba.co.kr/job/Detail.asp?adid=97215947
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97141384
http://www.alba.co.kr/job/Detail.asp?adid=97169231
http://www.alba.co.kr/job/Detail.asp?adid=97141338
http://www.alba.co.kr/job/Detail.asp?adid=97141318
http://www.alba.co.kr/job/Detail.asp?adid=97141250
http://www.alba.co.kr/job/Detail.asp?adid=97141229
http://www.alba.co.kr/job/Detail.asp?adid=97141212
http://www.alba.co.kr/job/Detail.asp?adid=97140839
http://www.alba.co.kr/job/Detail.asp?adid=97140599
http://www.alba.co.kr/job/Detail.asp?adid=97154970
http://www.alba.co.kr/job/Detail.asp?adid=96948898
http://www.alba.co.kr/job/Detail.asp?adid=97140535
http://www.alba.co.kr/job/Detail.asp?adid=97162274
http://www.alba.co.kr/job/Detail.asp?adid=97140001
http://www.alba.co.kr/job/Detail.asp?adid=97158921
http://www.alba.co.kr/job/Detail.asp?adid=97126571
http://www.alba.co.kr/job/Detail.asp?adid=97151178
http://www.alba.co.kr/job/Detail.asp?adid=97185338
http://www.alba.co.kr/job/Detail.asp?adid=91446418
http://www.alba.co.kr/job/Detai

http://www.alba.co.kr/job/Detail.asp?adid=97093603
http://www.alba.co.kr/job/Detail.asp?adid=97111562
http://www.alba.co.kr/job/Detail.asp?adid=96977969
http://www.alba.co.kr/job/Detail.asp?adid=96977961
http://www.alba.co.kr/job/Detail.asp?adid=97097934
http://www.alba.co.kr/job/Detail.asp?adid=97097917
http://www.alba.co.kr/job/Detail.asp?adid=97097908
http://www.alba.co.kr/job/Detail.asp?adid=97097899
http://www.alba.co.kr/job/Detail.asp?adid=97097892
http://www.alba.co.kr/job/Detail.asp?adid=97097883
http://www.alba.co.kr/job/Detail.asp?adid=97097876
http://www.alba.co.kr/job/Detail.asp?adid=97097807
http://www.alba.co.kr/job/Detail.asp?adid=97097808
http://www.alba.co.kr/job/Detail.asp?adid=97097813
http://www.alba.co.kr/job/Detail.asp?adid=97097810
http://www.alba.co.kr/job/Detail.asp?adid=97097811
http://www.alba.co.kr/job/Detail.asp?adid=97097812
http://www.alba.co.kr/job/Detail.asp?adid=97097809
http://www.alba.co.kr/job/Detail.asp?adid=97091960
http://www.alba.co.kr/job/Detai

In [12]:
df.head(25)

,알바명,알바설명,지역,지역세부,급여종류,금액,시간,주소
0,와이씨 컴퍼니(직영5팀),"홈쇼핑/택배 야간물류센터 상,하차 인원모집 최대15만원 당일지급",대구,중구,일급,153732,19:00~05:00,http://www.alba.co.kr/job/Detail.asp?adid=97438965
1,(주)한성엠에스,[12/16~20 주간] 콤프케이스 생산 단기 아르바이트(12/27지급),경남,창원시,일급,100000,08:00~19:30,http://www.alba.co.kr/job/Detail.asp?adid=97436358
2,(주)이학,이학갈비 연수점 대학생 아르바이트,인천,연수구,시급,8350,12:00~21:00,http://www.alba.co.kr/job/Detail.asp?adid=97431487
3,호텔신라,"[시급10,308원] 신라호텔 설특판 콜센터 근무자 모집(주휴포함)",서울,중구,시급,10308,09:00~18:00,http://www.alba.co.kr/job/Detail.asp?adid=97360731
4,호텔신라,"[시급10,308원] 신라호텔 설특판 콜센터 근무자 모집(주휴포함)",서울,강남구,시급,10308,09:00~18:00,http://www.alba.co.kr/job/Detail.asp?adid=97360734
5,호텔신라,"[시급10,308원] 신라호텔 설특판 콜센터 근무자 모집(주휴포함)",서울,용산구,시급,10308,09:00~18:00,http://www.alba.co.kr/job/Detail.asp?adid=97360733
6,호텔신라,"[시급10,308원] 신라호텔 설특판 콜센터 근무자 모집(주휴포함)",서울,종로구,시급,10308,09:00~18:00,http://www.alba.co.kr/job/Detail.asp?adid=97360732
7,와이씨 컴퍼니(직영5팀),"홈쇼핑/택배 야간물류센터 상,하차 인원모집 최대15만원 당일지급",경북,구미시,일급,153732,19:00~05:00,http://www.alba.co.kr/job/Detail.asp?adid=97438911
8,와이씨 컴퍼니(직영5팀),"홈쇼핑/택배 야간물류센터 상,하차 인원모집 최대15만원 당일지급",서울,동작구,일급,153732,19:00~05:00,http://www.alba.co.kr/job/Detail.asp?adid=97438906
9,호텔신라,"[시급11,000원]신라호텔 배송도우미 모집(주휴포함)",서울,송파구,시급,11000,08:00~17:00,http://www.alba.co.kr/job/Detail.asp?adid=97360652


### 상세페이지 크롤링을 진행했으나, 알바천국은 크롤링 차단이 엄격해서 잘 되지 않았습니다. 
### 'albaheaven_info_specific.csv'에서 볼 수 있듯이, 25페이지 정도만 크롤링되고 이후로는 바로 차단됩니다.
### 그래서 알바천국은 상세페이지 정보는 쓰지 않았습니다.